In [1]:
import re

import lyrics as ly
import json
from pathlib import Path


import spacy
nlp = spacy.load("fr_core_news_sm")

In [2]:
vald = ly.Artiste("Vald")
vald_songs = vald.songs
vald_songs


{<Song Freestyle téléphoné by Vald>,
 <Song Paradoxe by Vald>,
 <Song Freestyle Daymolition (27 août 2011) by Vald>,
 <Song Freestyle Daymolition A capella by Vald>,
 <Song Freestyle Daymolition by Vald>,
 <Song Poésie moderne by Vald>,
 <Song CQFD by Vald>,
 <Song Journal perso by Vald>,
 <Song Branleur by Vald>,
 <Song Deuspi by Vald (Ft. Suikon Blaz AD)>,
 <Song Elyxir by Vald (Ft. Georgio)>,
 <Song Fado by Vald>,
 <Song Gouaould by Vald>,
 <Song Guedin by Vald>,
 <Song Interlude by Vald>,
 <Song J'ai rêvé d'un verre de sang by Vald>,
 <Song Lalala by Vald>,
 <Song Lettre à Élise by Vald>,
 <Song Lune by Vald>,
 <Song Métafion by Vald (Ft. Suikon Blaz AD)>,
 <Song Pas grave by Vald>,
 <Song Rockstar by Vald>,
 <Song Smiley by Vald>,
 <Song Surprise by Vald>,
 <Song Teh stylé by Vald>,
 <Song Winston by Vald>,
 <Song Évoluchienne by Vald>,
 <Song Freestyle séquestration by Vald (Ft. Georgio)>,
 <Song Freestyle by Vald (Ft. Georgio)>,
 <Song Rendez-moi by Vald (Ft. Suikon Blaz AD)>,
 

In [3]:
lexiques = Path("lexiques").glob("*.json")

dict_lexiques = {
    fic.stem: set(json.load(fic.open(encoding="utf-8")))
    for fic in lexiques
}

lexique_ultime = set.union(*dict_lexiques.values())

# len(lexique_ultime) # 3 851 515 formes

In [4]:
def preclean(chaine):
    chaine = chaine.replace(u"\xa0", " ")
    chaine = chaine.replace(u"\u2009", " ")
    chaine = chaine.replace(u"\u200b", " ")
    chaine = chaine.replace(u"\u200c", " ")
    return chaine.replace(u"\u200d", " ")

def clean_word(word):
    try:
        word = word.strip()
    except:
        print(word)
        return
    word = word.strip(".,;“…’:!”?\"'()[]{}«»×*")
    if re.fullmatch(r"((\\x)|(\\u)|(\\n)|(\d+)).*", word):
        return
    return word

In [11]:
from spacy.tokenizer import Tokenizer
from spacy.lang.fr import French
nlp = French()

# Create a blank Tokenizer with just the French vocab
tokenizer = nlp.tokenizer

def tokenize(chaine):
    return [token.text for token in tokenizer(chaine)]

tokenize("C'est l'été, on va à la plage !")

["C'", 'est', "l'", 'été', ',', 'on', 'va', 'à', 'la', 'plage', '!']

In [6]:
neologismes = set()

for song in vald_songs:
    if song.paroles:
        paroles = tokenize(preclean(song.paroles))
        for word in paroles:
            word = clean_word(word)
            if word:
                if word.lower() not in lexique_ultime:
                    neologismes.add(word)

len(neologismes)


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


3791

In [7]:
neologismes


{"j'aurai",
 'tchoin',
 "m'as",
 "d'enfer",
 "l'double",
 "l'stock",
 "n'prends",
 'dutchs',
 "j'entends",
 'véritableJe',
 "t'épargne",
 "d'C",
 'd’faire',
 "j'prépare",
 'qu’elle',
 "l'hôtel",
 "j'dirais",
 'Joffrey',
 'j’irai',
 'Wow',
 "s'attrape",
 "d'en",
 "j'vomis",
 'n’remarque',
 "m'aimer",
 "c'toubab",
 'l’honneur',
 'qu’est',
 'd’autre',
 "J'baise",
 "d'vant",
 "l'oublier",
 'fonce-dé',
 "d'abonnés",
 's’approcha',
 "J'éteins",
 "d'embauche",
 'vas-go',
 "J'peux",
 "est-c'que",
 't’as',
 "m'a",
 "d'envies",
 "d'déhanchés",
 "d'vue",
 'X2',
 'dis-moi',
 "s'tapait",
 "J'voudrais",
 "J'tourne",
 "d'roue",
 "l'atterrissage",
 "d'enjeux",
 "j'trouve",
 'snobbé',
 'refrés',
 "d'grandes",
 "l'arrache",
 'tier-quar',
 "J'tape",
 'vever',
 "l'salaire",
 'colectés',
 "l'garrot",
 "l'Sheitan",
 'J’recherche',
 'Haze',
 "l'accorde",
 'qu’in-',
 "l'coquillage",
 'Suce-moi',
 "L'occident",
 'rhum-citronnade',
 "d'aller",
 "t'dévier",
 "qu'l'Afrique",
 'tasse-pé',
 'Montre-moi',
 "D't'faço